In [1]:
import shutil
import sys
import os
from pathlib import Path
import json
import re
import pysubs2
from pysubs2 import SSAFile, SSAEvent, make_time

In [2]:
root_path = Path('/home/jianghui/dataset/Hierachal-MovieNet/')
img_root_path = Path(root_path, 'all/img/')
shot_root_path = Path(root_path, 'all/shot/')
scene_root_path = Path(root_path, 'all/scene/')
subtitle_root_path = Path(root_path, 'all/subtitle/')
subtitle_frame_root_path = Path(root_path, 'all/subtitle_frame/')
subtitle_img_root_path = Path(root_path, 'all/subtitle_img/')
frame_root_path = Path(root_path, 'all/frame/')
synopsis_root_path = Path(root_path, 'all/synopsis/')
cast_root_path = Path(root_path, 'all/cast/')
names = locals()

In [3]:
tt_list = []
with open(Path(root_path, 'movie_id_list'), 'r') as f_tt_list:
    for tt_line in f_tt_list:
        tt_list.append(tt_line.replace('\n', ''))

In [4]:
def open_json(types:str, tt_id):
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'r') as names[f'f_{types}']:
        names[f'{types}_json'] = json.load(names[f'f_{types}'])
    return names[f'{types}_json']

def save_json(types:str, tt_id):
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'w') as names[f'f_{types}']:
        json.dump(names[f'{types}_json'], names[f'f_{types}'], indent=2)

In [5]:
old_root_path = Path("/home/jianghui/dataset/MovieNet/")
srt_root_path = Path(old_root_path, "subtitle1K.v1/subtitle/")
shot_det_root_path = Path(old_root_path, "movie1K.shot_detection.v1/shot/")

In [6]:
info_path = Path(old_root_path, "movie1K.video_info.v1.json")
with open(info_path, 'r') as f_info:
    info_json = json.load(f_info)

In [39]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    
    img_frame_json = []

    with open(Path(shot_det_root_path, f'{tt_id}.txt'), 'r') as f_shot_det:
        for shot_id, line in enumerate(f_shot_det):
            line_list = line.split()
            for frame_id, frame in enumerate(line_list[2:]):
                img_frame_json_ele = {
                    "img_id" : f'{shot_id}_{frame_id}',
                    "frame" : int(frame)
                }
                img_frame_json.append(img_frame_json_ele)
    
    with open(Path(frame_root_path, f'{tt_id}.json'), 'w') as f_frame:
        json.dump(img_frame_json, f_frame, indent=2)

    

            

In [34]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    subtitle_frame_json = []

    srt_file_path = Path(srt_root_path, f'{tt_id}.srt')
    #srt_file = SSAFile.load(srt_file_path)

    fps = info_json[tt_id]["fps"]

    srt_json = []
    try:
        srt_file = SSAFile.load(srt_file_path, encoding='utf-8')
    except:
        try:
            srt_file = SSAFile.load(srt_file_path, encoding='ISO-8859-1')
        except:
            srt_file = SSAFile.load(srt_file_path, encoding='utf-16')
    
    for srt in srt_file:
        start_frame = pysubs2.time.ms_to_frames(srt.start, fps)
        end_frame = pysubs2.time.ms_to_frames(srt.end, fps)
        srt_json_ele = {
                "frame" : [start_frame, end_frame],
                "sentence" : srt.plaintext.replace("\n"," ")
            }
        srt_json.append(srt_json_ele)
    
    with open(Path(subtitle_frame_root_path, f'{tt_id}.json'), "w") as f_subtitle_frame:
        json.dump(srt_json, f_subtitle_frame, indent=2)



processing tt0047396 ...
processing tt0048545 ...
processing tt0049730 ...
processing tt0052357 ...
processing tt0056923 ...
processing tt0061418 ...
processing tt0061722 ...
processing tt0062622 ...
processing tt0063442 ...
processing tt0065214 ...
processing tt0065724 ...
processing tt0066026 ...
processing tt0067116 ...
processing tt0068646 ...
processing tt0070735 ...
processing tt0071315 ...
processing tt0071562 ...
processing tt0072684 ...
processing tt0073195 ...
processing tt0074285 ...
processing tt0075148 ...
processing tt0076759 ...
processing tt0077405 ...
processing tt0078788 ...
processing tt0078841 ...
processing tt0079417 ...
processing tt0080684 ...
processing tt0081505 ...
processing tt0082089 ...
processing tt0082971 ...
processing tt0083658 ...
processing tt0083929 ...
processing tt0084726 ...
processing tt0086190 ...
processing tt0086250 ...
processing tt0086879 ...
processing tt0087332 ...
processing tt0088847 ...
processing tt0088944 ...
processing tt0090022 ...


In [61]:
a = "5:00 p.m., Jardins des Champs-\u00c3\u2030lys\u00c3\u00a9es"
a

'5:00 p.m., Jardins des Champs-Ã‰lysÃ©es'

In [54]:
a = "- Bertha, you may serve the souffl\u00e9. - Yes, ma'am."
re.sub(r'\\u.{4} ', '', a.encode('raw_unicode_escape').decode('utf-8'))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 34: invalid continuation byte

In [62]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    frame_json = open_json('frame', tt_id)
    subtitle_frame_json = open_json('subtitle_frame', tt_id)

    subtitle_img_json = []

    subtitle_frame_json_begin_idx = 0
    matched_list = [False for i in range(len(subtitle_frame_json))]

    for frame_json_ele in frame_json:
        subtitle_img_json_ele = {
            "img_id" : frame_json_ele["img_id"],
            "sentence" : []
        }
        flag = False
        for idx, subtitle_frame_json_ele in enumerate(subtitle_frame_json):
            if (frame_json_ele["frame"] >= subtitle_frame_json_ele["frame"][0]) and (frame_json_ele["frame"] <= subtitle_frame_json_ele["frame"][1]):
                if not matched_list[idx]:
                    subtitle_img_json_ele["sentence"].append(subtitle_frame_json_ele["sentence"])
                    subtitle_img_json.append(subtitle_img_json_ele)
                    matched_list[idx] = True
                    flag = True
                    break
                else:
                    flag = True
                    break
            
        if not flag:
            subtitle_img_json.append(subtitle_img_json_ele)
    
    save_json("subtitle_img", tt_id)


processing tt0047396 ...
processing tt0048545 ...
processing tt0049730 ...
processing tt0052357 ...
processing tt0056923 ...
processing tt0061418 ...
processing tt0061722 ...
processing tt0062622 ...
processing tt0063442 ...
processing tt0065214 ...
processing tt0065724 ...
processing tt0066026 ...
processing tt0067116 ...
processing tt0068646 ...
processing tt0070735 ...
processing tt0071315 ...
processing tt0071562 ...
processing tt0072684 ...
processing tt0073195 ...
processing tt0074285 ...
processing tt0075148 ...


KeyboardInterrupt: 